In [ ]:
pip install scikit-learn pickle-mixin bluetarget

# Saving Model

To begin with BlueTarget, you will need to save your trained models.

In [ ]:
from sklearn import svm
from sklearn import datasets

import pickle

# Load training data set
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Train the model
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

pickle.dump(clf, open('./model.pkl', 'wb'))


# Creating a service

Services are the core components of BlueTarget, where the serving logic is defined. Create a file name service.py with:

In [ ]:
%%writefile service.py
import os
from typing import Dict, List

class Model:
    def __init__(self) -> None:
        self._model = None

    def load(self):
        import pickle

        with open(f"{os.path.dirname(__file__)}/model.pkl", 'rb') as pickle_file:
            self._model = pickle.load(pickle_file)

    def predict(self, request: Dict) -> Dict[str, List]:
        response = {}
        inputs = request["inputs"]
        result = self._model.predict(inputs).tolist()
        response["predictions"] = result

        return response

Writing service.py


# Create requirement file
Here is where you define your model requirements, in our case we have  scikit-learn pickle-mixin dependencies

In [ ]:
%%writefile requirements.txt
scikit-learn==1.0.2
pickle-mixin==1.0.2

Writing requirements.txt


# Create a deployment file

Here you define the deployment configuration, you also can integrate this logic within your deployment pipeline.

You can get your API KEY cliking [here](https://deploy.bluetarget.ai/api-keys) 


In [ ]:
from bluetarget import BlueTarget

bt = BlueTarget(api_key="YOUR_API_KEY")

bt.deploy(
    model_name="FirstModel",
    model_class="Model",
    model_files=["service.py", "model.pkl"],
    requirements_file="requirements.txt"
)